In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
using Statistics
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")

In [ ]:
#load borders
border = import_border("../data/border.json")

# create the lattice grid
dx = 40
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))

# define x and y coordinate for plotting
x = repeat(reshape(xrange,1,Nx), Ny, 1)
y = repeat(reshape(yrange,Ny,1), 1, Nx);

In [ ]:
# compute or load grid parameters
recompute_param = true

if(recompute_param || isfile("../numerics/grid_params_" * string(dx) * ".h5"))
    bx, by, p, m, d = get_params_diff(isinside, isborder, Float64(dx), yrange, xrange, "../data/pantagruel.h5",
        "../numerics/grid_params_" * string(dx) * ".h5", Niter=5000, dmax = 10.0, patch=0.001, bmin=0.01)
else
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5")
end

In [ ]:
trapz((yrange,xrange), m)
#contour(m,fill=true)

In [ ]:
# plot some physical quantities
plot(do_plot(isinside, p), do_plot(isinside, m), do_plot(isinside, d), layout=(3,1), size=(500,900))


In [ ]:
println(minimum(m[isinside]))
println(minimum(d[isinside]))
#bx ./= 8000
#by ./= 8000
#bx ./= 4
#by ./= 4
#bx .= max.(bx,1)
#by .= max.(by,1)
contour(by,fill=true)
#contour(by)

In [ ]:
# compute, iteratively, the stable solution
interval = 1000
Niter = 15000
th = zeros(Ny, Nx)

@time begin
    for k in 1:Niter
        if(mod(k,interval) == 0)
            temp = copy(th)
        end
        Threads.@threads for i in 2:Ny-1
            Threads.@threads for j in 2:Nx-1
                if(isinside[i,j])
                    bij = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1])
                    th[i,j] = (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] + 
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1] + dx^2*p[i,j]) / bij
                end
            end
        end
        Threads.@threads for k in 1:size(n,1)
            i = Int64(n[k,1])
            j = Int64(n[k,2])
            nx = n[k,4]
            ny = n[k,3]
            if(nx == 1)
                th[i,j] = th[i,j-2]
            elseif(nx == -1)
                th[i,j] = th[i,j+2]
            end
            if(ny == 1)
                th[i,j] = th[i-2,j]
            elseif(ny == -1)
                th[i,j] = th[i+2,j]
            end
        end  
        if(mod(k,interval) == 0)
            println([k maximum(abs.(th-temp))])
        end

    end
end


In [ ]:
# plot the result
do_plot(isinside, th)
#contour(bx)

In [ ]:
th2 = set_ref_phase(isinside, yrange, xrange, th, [741.57,-1078.62])

x = repeat(reshape(xrange, 1, Nx), Ny, 1)
y = repeat(reshape(yrange, Ny, 1), 1, Nx)
println(size(x))
println(size(y))
#scatter(x[isinside], y[isinside])
do_plot(isinside, th2)
savefig("cont_th.pdf")

In [ ]:
# save the result
fid = h5open("../numerics/stable_" * string(dx) * ".h5", "w")
write(fid, "th", th)
close(fid)

In [ ]:
 gen, dem, bline, idb, idgen, coord, mg, dg, dl, th_ref = load_discrete_model("../data/pantagruel.h5");
println(coord[1,:])
#scatter(coord[:,1], coord[:,2])

In [ ]:
th_disc = get_discrete_values(yrange, xrange, th, coord);

In [ ]:
scatter(th_disc)
scatter!(th_ref)

In [ ]:
contour(get_cont_values(isinside, yrange, xrange, coord, th_ref),fill=true)
savefig("disc_th.pdf")

In [ ]:
fx, fy = power_flow(isinside, th, bx, by, Float64(dx))

In [ ]:
contour(fx, fill=true)

In [ ]:
contour(fy, fill=true)

In [ ]:

quiver(x[isinside], y[isinside], quiver=(200*fx[isinside], 200*fy[isinside]))
savefig("asdf.svg")